Desired coordinate frames:

![coordinate frames](https://i.imgur.com/9Dn5Dfl.png)

In [ ]:
import numpy as np
from cloth_tools.config import load_camera_pose_in_left_and_right


X_LCB_C, X_RCB_C = load_camera_pose_in_left_and_right()

In [ ]:
X_LCB_RCB = X_LCB_C @ np.linalg.inv(X_RCB_C)
with np.printoptions(precision=3, suppress=True):
    print("Pose of the right robot base in the left robot base frame: X_LCB_RCB =")
    print(X_LCB_RCB)

In [ ]:
p_LCB_RCB = X_LCB_RCB[:3, 3]
with np.printoptions(precision=3, suppress=True):
    print(p_LCB_RCB)

In [ ]:
print("Distance between the two robot bases (according to the camera calibration):")
np.linalg.norm(p_LCB_RCB)

In [ ]:
# We define the world origin as the middle point between the two robot bases
p_LCB_W = p_LCB_RCB / 2.0
with np.printoptions(precision=3, suppress=True):
    print(p_LCB_W)

In [ ]:
from airo_spatial_algebra.se3 import SE3Container

# The line below define the world Y-axis as the vector from the right robot base to the left robot base
y_LCB_W = -p_LCB_W / np.linalg.norm(p_LCB_W)
z_LCB_W = np.array([0, 0, 1])
x_LCB_W = np.cross(y_LCB_W, z_LCB_W)
x_LCB_W = x_LCB_W / np.linalg.norm(x_LCB_W)
z_LCB_W = np.cross(x_LCB_W, y_LCB_W)  # make sure z-axis is orthogonal to x and y


R_LCB_W = np.column_stack((x_LCB_W, y_LCB_W, z_LCB_W))

X_LCB_W = np.identity(4)
X_LCB_W[:3, :3] = R_LCB_W
X_LCB_W[:3, 3] = p_LCB_W

X_W_LCB = np.linalg.inv(X_LCB_W)
X_W_RCB = X_W_LCB @ X_LCB_RCB

with np.printoptions(precision=3, suppress=True):
    print("Pose of the left robot base in the world frame: X_W_LCB =")
    print(X_W_LCB)
    print("Pose of the right robot base in the world frame: X_W_RCB =")
    print(X_W_RCB)

In [ ]:
# bringing it all together
from airo_typing import HomogeneousMatrixType


X_LCB_C, X_RCB_C = load_camera_pose_in_left_and_right()


def create_egocentric_world_frame(
    camera_pose_in_left: HomogeneousMatrixType, camera_pose_in_right: HomogeneousMatrixType
) -> tuple[HomogeneousMatrixType:, HomogeneousMatrixType, HomogeneousMatrixType]:
    X_LCB_C = camera_pose_in_left
    X_RCB_C = camera_pose_in_right
    X_LCB_RCB = X_LCB_C @ np.linalg.inv(X_RCB_C)
    p_LCB_RCB = X_LCB_RCB[:3, 3]

    # Define the world origin as the middle point between the two robot bases
    p_LCB_W = p_LCB_RCB / 2.0

    # Define the world Y-axis as the vector from the right robot base to the left robot base
    y_LCB_W = -p_LCB_RCB / np.linalg.norm(p_LCB_RCB)

    # Define (for now) that the world Z-axis is the same as the left_base Z-axis
    # We will fix later that it is not exactly orthogonal to the world Y-axis
    z_LCB_W = np.array([0, 0, 1])

    # Define the world X-axis as the cross product of the world Y and Z axes
    x_LCB_W = np.cross(y_LCB_W, z_LCB_W)
    x_LCB_W = x_LCB_W / np.linalg.norm(x_LCB_W)  # normalize because Z was not exactly orthogonal to Y

    # Ensure the world Z-axis is orthogonal to the world X and Y axes
    z_LCB_W = np.cross(x_LCB_W, y_LCB_W)

    # Create the rotation matrix
    R_LCB_W = np.column_stack((x_LCB_W, y_LCB_W, z_LCB_W))

    # Create the homogeneous transformation matrix
    X_LCB_W = np.identity(4)
    X_LCB_W[:3, :3] = R_LCB_W
    X_LCB_W[:3, 3] = p_LCB_W

    X_W_LCB = np.linalg.inv(X_LCB_W)
    X_W_RCB = X_W_LCB @ X_LCB_RCB

    X_W_C = X_W_LCB @ X_LCB_C
    # X_W_C_ = X_W_RCB @ X_RCB_C # Alternative, but should give exactly the same result I belive
    return X_W_C, X_W_LCB, X_W_RCB


X_W_C, X_W_LCB, X_W_RCB = create_egocentric_world_frame(X_LCB_C, X_RCB_C)

with np.printoptions(precision=3, suppress=True):
    print("Pose of the camera in the world frame: X_W_C =")
    print(X_W_C)
    print("Pose of the left robot base in the world frame: X_W_LCB =")
    print(X_W_LCB)
    print("Pose of the right robot base in the world frame: X_W_RCB =")
    print(X_W_RCB)